In [3]:
from spark.spark_config import create_spark_session
spark = create_spark_session()

In [4]:
from data_loader import DataLoader
from preprocessor import Preprocessor
from nlp_processor import NLPProcessor
from feature_extractor import FeatureExtractor
from eda_analyser import EdaAnalyser
from repository.cassandra_provider import CassandraProvider

In [5]:
# Load file
data_loader = DataLoader(spark)
df_full = data_loader.load_raw_csv()
df = df_full.limit(100000)

In [6]:
# # EDA
# analyser = EdaAnalyser(df)
# analyser.run_full_eda_report(['title', 'lyrics', 'views'])

In [7]:
df = Preprocessor.run(df)

In [8]:
df_tokenized = NLPProcessor.run(df)
df_tokenized.cache()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [9]:
extractor = FeatureExtractor()
extractor.fit(df_tokenized)
df_final = extractor.transform(df_tokenized)

In [10]:
cassandra_provider = CassandraProvider()
cassandra_provider.save(df_final)

In [11]:
spark.stop()